# **Project 2**
### Josh Winnes and Jon Contreras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import geopandas as gpd
import shapely
import skmob
from skmob.measures.individual import jump_lengths
from skmob.tessellation import tilers
from skmob.preprocessing import detection, clustering
from skmob.preprocessing import filtering
from skmob.preprocessing import compression

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

import os
import sys

try:       
    kmz_file_path_josh = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    kmz_file_path_jon = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    gpx_file_name_josh = 'March 7, 2024 - Whistler Blackcomb(Josh).gpx'
    gpx_file_name_jon = 'March 7, 2024 - Whistler Blackcomb(Jon).gpx'
    script_directory = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/Project2'
    
    os.chdir('/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data')
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    gpx_josh, run_info_josh = data_preprocessing(gpx_file_name_josh, kmz_file_path_josh)
    gpx_jon, run_info_jon = data_preprocessing(gpx_file_name_jon, kmz_file_path_jon)
    
    print('Working from Josh\'s laptop')
except FileNotFoundError:
    
    data_dir = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 2 Spatial/Spacial Project'
    kmz_file_path_josh = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 2 Spatial/Spacial Project/Joshwhistler.kmz'
    kmz_file_path_jon = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 2 Spatial/Spacial Project/jonwhistler.kmz'
    gpx_file_name_josh = 'Josh Mar 7, 2024 - Whistler Blackcomb.gpx'
    gpx_file_name_jon = 'Jon Mar 7, 2024 - Whistler Blackcomb.gpx'
    script_directory = '/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Project2git'
    
    os.chdir('/Users/jonathancontreras/Desktop/College Folder/Senior Year/Math485/Unit 2 Spatial/Spacial Project')
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    gpx_josh, run_info_josh = data_preprocessing(gpx_file_name_josh, kmz_file_path_josh)
    gpx_jon, run_info_jon = data_preprocessing(gpx_file_name_jon, kmz_file_path_jon)
    
    print('Working from Jonathan\'s laptop')

Working from Jonathan's laptop


In [2]:
gpx_jon2 = gpx_jon
gpx_josh2 = gpx_josh

files = [gpx_jon, gpx_josh]
for file in files:
    file_gdf = gpd.GeoDataFrame(file)
    file_gdf["geometry"] = gpd.points_from_xy(file_gdf.Longitude, file_gdf.Latitude)
    file_trajectory = skmob.TrajDataFrame(file, latitude='Latitude', longitude='Longitude', \
         datetime='Time', \
    )
    file.insert(loc = 2, column = "timestamp_UTC",
                value = file['Time'].apply(lambda x: x.tz_localize(tz='UTC'))
                )
    file.sort_values(by = ["Name", "Time"], inplace=True)
    file['year'] = file["Time"].dt.year
    file['month'] = file["Time"].dt.month
    file['date'] = file["Time"].dt.date

In [3]:
files = [gpx_jon, gpx_josh]
trajectory = {}
geo_dataframes = {}

for file, name in zip(files, ['gpx_jon', 'gpx_josh']):
    # Create and store GeoDataFrame
    file_gdf = gpd.GeoDataFrame(file)
    file_gdf["geometry"] = gpd.points_from_xy(file_gdf.Longitude, file_gdf.Latitude)
    geo_dataframes[name] = file_gdf
    
    # Create and store TrajDataFrame
    file_trajectory = skmob.TrajDataFrame(file, latitude='Latitude', longitude='Longitude', datetime='Time')
    trajectory[name] = file_trajectory

# Now you can access, for example:
# trajectory['gpx_jon']
# geo_dataframes['gpx_josh']


In [4]:
files = [gpx_jon, gpx_josh]
for file in files:
    # Step 1: Copy the original 'Time' column into a new column 'Time_original'
    file["Time_Pacific"] = file["Time"]
    
    # Step 2: Convert 'Time' column to UTC
    file["Time_UTC"] = file["Time"].dt.tz_convert('UTC')
    
    # Optional: Convert the original 'Time' column back to the original timezone if desired
    # Replace 'US/Pacific' with the original timezone if it was different
    file["Time"] = file["Time"].dt.tz_convert('US/Pacific')


In [5]:
gpx_jon.head()

,Latitude,Longitude,Elevation,Time,Name,Time_Pacific,Time_UTC
0,50.111385,-122.951516,699.522521,2024-03-07 09:52:51-08:00,pre_lift_1,2024-03-07 09:52:51-08:00,2024-03-07 17:52:51+00:00
1,50.109740,-122.953332,709.858664,2024-03-07 09:52:52-08:00,pre_lift_1,2024-03-07 09:52:52-08:00,2024-03-07 17:52:52+00:00
2,50.109765,-122.953274,741.954162,2024-03-07 09:52:53-08:00,pre_lift_1,2024-03-07 09:52:53-08:00,2024-03-07 17:52:53+00:00
3,50.109679,-122.953279,746.486242,2024-03-07 09:52:55-08:00,Lift 1,2024-03-07 09:52:55-08:00,2024-03-07 17:52:55+00:00
4,50.109586,-122.953275,748.005084,2024-03-07 09:52:58-08:00,Lift 1,2024-03-07 09:52:58-08:00,2024-03-07 17:52:58+00:00


In [6]:
import osmnx as ox
place_name = "Whistler, Canada" #Isla Santa Cruz, Galapagos, Ecuador
whistler_gdf = ox.geocode_to_gdf(place_name)
tessellation = tilers.tiler.get("h3_tessellation", base_shape=whistler_gdf,meters =200)
tessellation["tile_lng"] = tessellation.geometry.centroid.x
tessellation["tile_lat"] = tessellation.geometry.centroid.y

In [7]:
jon_trajectory = trajectory['gpx_jon']

gpx_jon_jl = jump_lengths(jon_trajectory)
gpx_jon_jl["jump_lengths"] = \
    gpx_jon_jl["jump_lengths"].apply(
        lambda x: np.append(x,np.nan).tolist())
gpx_jon_jl = gpx_jon_jl.explode(["jump_lengths"],
                ignore_index=True).reset_index(drop=True)
gpx_jon["jump_length"] = gpx_jon_jl.jump_lengths
trajectory['gpx_jon']["jump_length"] = gpx_jon_jl["jump_lengths"]

In [43]:
jon_trajectory_mapping = jon_trajectory.mapping(tessellation,remove_na =True) # map trajectory into tessellation tiles
gpx_jon_tiled = jon_trajectory_mapping.merge(tessellation, on = "tile_ID", how = "inner") # join to get the tile coordinates
gpx_jon_tiled.drop(columns =["lat","lng"], inplace=True) # drop the original lat/lng so that we can create new trajectory
gpx_jon_tiled.sort_values(by = ["Name", "datetime"], inplace=True) # sort before making trajectory
gpx_jon_tiled_trajectory = skmob.TrajDataFrame(gpx_jon_tiled, latitude='tile_lat', longitude='tile_lng', \
    user_id='Name', datetime='Time_UTC', \
)
jon_trajectory_compressed = compression.compress(jon_trajectory, spatial_radius_km=0.02) # compressed trajectory
gpx_jon_tiled_trajectory_compressed = compression.compress(gpx_jon_tiled_trajectory, spatial_radius_km=0.02) # compressed trajectory
gpx_jon_stay_location = detection.stay_locations(jon_trajectory, 
    stop_radius_factor=0.02, minutes_for_a_stop=1, spatial_radius_km=0.2, leaving_time=True)

In [44]:
color_list = ['#883399','#66c066', '#ff800c','#60ff78','#301e4b'] # purple, green, orange, lime, dark purple
color_list.extend(list(sns.color_palette('tab10').as_hex()))
color_list = color_list * 3 # extend the list by repeating 3 times


In [45]:
jon_plot = jon_trajectory.plot_trajectory(
    hex_color=color_list[0],weight=1, zoom=14, opacity = 0.9,
    tiles = 'OpenStreetMap', 
    )
jon_trajectory_compressed.plot_trajectory(map_f = jon_plot, hex_color = color_list[1],weight = 1, opacity = 0.9)
gpx_jon_tiled_trajectory_compressed.plot_trajectory(map_f = jon_plot, hex_color = color_list[2], weight=1, opacity = 0.9)
jon_plot

In [46]:

jon_trajectory.plot_trajectory(tiles = "OpenStreetMap", zoom = 10)

In [48]:
# Initialize lists to store stay and trajectory segments
jon_stay_segment = []
jon_segment = []

# Use a single date range since we're focusing on one day
jon_stay_segment.append(gpx_jon_stay_location.query("datetime >= @s_date and datetime <= @e_date"))
jon_segment.append(jon_trajectory.query("datetime >= @s_date and datetime <= @e_date"))

# Extract stay coordinates for PolyLine, filtering out NaN values
jon_stay_coordinates = list(zip(gpx_jon_stay_location['lat'], gpx_jon_stay_location['lng']))
jon_stay_coordinates = [(lat, lng) for lat, lng in jon_stay_coordinates if pd.notna(lat) and pd.notna(lng)]

# Check if there are any valid coordinates left after filtering
if not jon_stay_coordinates:
    raise ValueError("No valid latitude and longitude coordinates available for plotting.")

# Calculate the mean latitude and longitude only from valid coordinates
mean_lat = np.mean([lat for lat, _ in jon_stay_coordinates])
mean_lng = np.mean([lng for _, lng in jon_stay_coordinates])

# Initialize the map centered around the mean location
jon_plot = folium.Map(location=[mean_lat, mean_lng], zoom_start=13)

# Add circle markers for each stay location, ensuring they are valid
for lat, lng, leave_time in zip(gpx_jon_stay_location['lat'], gpx_jon_stay_location['lng'], gpx_jon_stay_location['leaving_datetime']):
    if pd.notna(lat) and pd.notna(lng):  # Check if lat and lng are valid
        year = leave_time.year  # Extract year from leave_time
        year_offset = year - 2011  # Adjust year for color indexing
        folium.CircleMarker(
            location=[lat, lng],
            tooltip=leave_time.date(),
            radius=5,
            weight=1,
            color=color_list[year_offset]  # Ensure color_list has enough colors
        ).add_to(jon_plot)

# Add PolyLine connecting stay locations
if jon_stay_coordinates:  # Only add PolyLine if there are valid coordinates
    folium.PolyLine(jon_stay_coordinates, weight=1).add_to(jon_plot)

# Display the map
jon_plot
